<a href="https://colab.research.google.com/github/cryssoga/DSWP/blob/master/fork_of_notebook1ca06f35f5_crys_atualizado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#df = pd.read_csv('/kaggle/input/labdata-churn-challenge-2020/train.csv')
#df.head()

In [35]:
# eu
import numpy as np # linear algebra
import pandas as pd

url_train = 'https://raw.githubusercontent.com/cryssoga/DSWP/master/Li%C3%A7%C3%A3o/train.csv'
url_test = 'https://raw.githubusercontent.com/cryssoga/DSWP/master/Li%C3%A7%C3%A3o/test.csv'
url_sample_submission = 'https://raw.githubusercontent.com/cryssoga/DSWP/master/Li%C3%A7%C3%A3o/sample_submission.csv'

df_train = pd.read_csv(url_train)
df_test = pd.read_csv(url_test)
df_sample_submission = pd.read_csv(url_sample_submission)


In [ ]:
df_sample_submission.shape

(1409, 2)

In [36]:
def cria_total_serv(df):
  total_serv = []
  col_reflexo = ['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']
  linha = 0
  while linha < df.shape[0]:
    qtde=0
    if df['InternetService'][linha]=='No':
      qtde = 0
    else:
      for i in col_reflexo:
        if df[i][linha] == 'Yes':
          qtde = qtde + 1
    total_serv.append(qtde)
    linha+=1
  df['total_serv']=total_serv

In [37]:
cria_total_serv(df_train)
cria_total_serv(df_test)

In [38]:
def f_trata_col(df):
    df['Dependents'].fillna('No', inplace=True)
    df['PaymentMethod'].fillna('Electronic check',inplace = True)
    df['tenure'].fillna(df['tenure'].median(),inplace = True)
    df['TotalCharges'] = df['TotalCharges'].replace(' ', '0')
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])
    df['TotalCharges'] = df['TotalCharges'].replace(0, df['TotalCharges'].median())
    df['SeniorCitizen'] = df['SeniorCitizen'].replace(0,'0')
    df['SeniorCitizen'] = df['SeniorCitizen'].replace(1,'1')
    ohe = pd.get_dummies(df)
    ohe=pd.get_dummies(df,drop_first=True)
    ohe.columns = ohe.columns.str.replace(' ', '_')
    ohe.columns = ohe.columns.str.replace('(','')
    ohe.columns = ohe.columns.str.replace(')','')
    return ohe

In [39]:
df_train = f_trata_col(df_train)

In [40]:
df_test = f_trata_col(df_test)

In [41]:
df_test.shape

(1409, 32)

In [42]:
X = df_train.drop(columns= 'Churn', axis= 1)
y = df_train['Churn']
X_test = df_test.copy()
y_test = df_sample_submission['Churn']

In [43]:
X.shape

(5634, 32)

MODELO: XGBoost

In [44]:
i_CV = 10 # Número de Cross-Validations
i_Seed = 22091980 # semente por questões de reproducibilidade
# f_Test_Size = 0.3 # Proporção do dataframe de validação (outros valores poderiam ser 0.15, 0.20 ou 0.25)

In [45]:
# from sklearn.model_selection import train_test_split

# X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(df_X, df_y, test_size = f_Test_Size, random_state = i_Seed)

y_teste = df_sample_submission['Churn']
X_treinamento = X.copy()
y_treinamento = y.copy()
X_teste = df_test.copy()

In [46]:
f'"X_treinamento.shape:" {X_treinamento.shape}, "y_treinamento_shape:"{y_treinamento.shape},"X_teste.shape:"{X_teste.shape},"y_teste.shape:"{y_teste.shape}'



'"X_treinamento.shape:" (5634, 32), "y_treinamento_shape:"(5634,),"X_teste.shape:"(1409, 32),"y_teste.shape:"(1409,)'

In [47]:
from xgboost import XGBClassifier
import xgboost as xgb

# Instancia...
ml_XGB= XGBClassifier(silent=False, 
                         scale_pos_weight=1,
                        learning_rate=0.01,  
                        colsample_bytree = 1,
                        subsample = 0.8,
                        objective='binary:logistic', 
                        n_estimators=1000, 
                        reg_alpha = 0.3,
                        max_depth= 3, 
                        gamma=1, 
                        max_delta_step=5)

# Treina... 
ml_XGB.fit(X_treinamento, y_treinamento)



XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=1,
              learning_rate=0.01, max_delta_step=5, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0.3, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=False, subsample=0.8, verbosity=1)

In [48]:
from sklearn.model_selection import cross_val_score

In [49]:
def funcao_cross_val_score(modelo, X_treinamento, y_treinamento, CV):
    #versão com cross_val_score::
    a_scores_CV = cross_val_score(modelo, X_treinamento, y_treinamento, cv = CV)
    print(f'Média das Acurácias calculadas pelo CV....: {100*round(a_scores_CV.mean(),4)}')
    print(f'std médio das Acurácias calculadas pelo CV: {100*round(a_scores_CV.std(),4)}')
    return a_scores_CV

In [50]:
a_scores_CV = funcao_cross_val_score(ml_XGB, X_treinamento, y_treinamento, i_CV)

Média das Acurácias calculadas pelo CV....: 80.9
std médio das Acurácias calculadas pelo CV: 2.03


In [51]:
y_pred = ml_XGB.predict(X_teste)

In [53]:
unique_elements, counts_elements = np.unique(y_pred, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[   0    1]
 [1107  302]]


In [52]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_teste, y_pred)
cf_matrix

array([[574, 160],
       [533, 142]])

In [59]:
df_submit = pd.DataFrame(zip(X_teste['id'],y_pred), columns = ['id','Churn'])

In [60]:
df_submit.to_csv('churn_01.csv',index = False, sep = ',')

In [62]:
df_submit.Churn.value_counts()

0    1107
1     302
Name: Churn, dtype: int64

In [ ]:
734/1105

0.6642533936651583

In [56]:
X_treinamento = X_treinamento.drop(['DeviceProtection_Yes','TotalCharges','gender_Male','PaymentMethod_Mailed_check',
                   'PaymentMethod_Credit_card_automatic', 'PhoneService_Yes','MultipleLines_No_phone_service',
                   'total_serv', 'OnlineBackup_Yes', 'Partner_Yes'],axis=1)
ml_XGB.fit(X_treinamento, y_treinamento)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=1,
              learning_rate=0.01, max_delta_step=5, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0.3, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=False, subsample=0.8, verbosity=1)

In [57]:
X_teste = X_teste.drop(['DeviceProtection_Yes','TotalCharges','gender_Male','PaymentMethod_Mailed_check',
                   'PaymentMethod_Credit_card_automatic', 'PhoneService_Yes','MultipleLines_No_phone_service',
                   'total_serv', 'OnlineBackup_Yes', 'Partner_Yes'],axis=1)
y_pred_2 = ml_XGB.predict(X_teste)

In [58]:
unique_elements, counts_elements = np.unique(y_pred_2, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[   0    1]
 [1097  312]]
